In [1]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook
#%texify

In [357]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
from pathlib import Path
import pickle
import operator
sys.setrecursionlimit(10000)
%matplotlib qt5

In [358]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.analyze_eval import *
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assemble_full_img_predictions

: 

: 

In [4]:
import scipy.signal
import skimage.io as sio

In [5]:
outputdir = '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_ter_pos01/fork_outputs/'

In [6]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/good_models/model_8865_venus_low_ph.pkl')
with open(model_path, 'rb') as f:
    model = pickle.load(f)

Save training fluorescence images and location of cell masks

In [7]:
venus_images = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_3041_ter_pos01/venus/')
venus_filenames = sorted(list(venus_images.glob('*.tiff')))

In [8]:
venus_filenames = venus_filenames

In [9]:
eval_imgs_all = []
for filename in venus_filenames:
    eval_imgs_all.append(sio.imread(filename))

In [10]:
eval_imgs_all = np.stack(eval_imgs_all)

In [11]:
eval_imgs_all.shape

(301, 1041, 1302)

In [12]:
txtfile = outputdir+'/'+ 'train_cell_masks.txt'
with open(txtfile, 'w') as f:
    for i in range(len(venus_filenames)):
         f.write(venus_filenames[i].as_posix()+"\n")
f.close()

Save predicted and true dot localizations on training data

In [13]:
eval2 = eval_imgs_all
fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
plot_num = None
preds_raw, n_per_img, plot_area = recognition(model=model, eval_imgs_all=eval2, batch_size=32, use_tqdm=False,
                                     nms=True, candidate_threshold=0.05,
                                     nms_threshold=0.05,
                                     pixel_nm=model.data_generator.psf_params['pixel_size_xy'],
                                     plot_num=1,
                                     win_size=model.data_generator.simulation_params['train_size'],
                                     padding=True,
                                     padded_background=model.simulation_params['bg_values'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [14]:
fov_size

[84630, 67665]

In [15]:
plot_full_img_predictions(model, plot_infs=plot_area, eval_csv=None,
                          plot_num=1, fov_size=fov_size,
                          pixel_size=[65, 65])

In [15]:
sigma_x = np.array([x[7] for x in preds_raw])
sigma_y = np.array([x[8] for x in preds_raw])
sigma_z = np.array([x[9] for x in preds_raw])
sigma_ph = np.array([x[10] for x in preds_raw])
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=100)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=100)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=100)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=50)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

fig.suptitle('Sigmas predicted by the network')
plt.tight_layout()
plt.show()

In [17]:
offsets_x = np.array([x[11] for x in preds_raw])
offsets_y = np.array([x[12] for x in preds_raw])

In [18]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].hist(offsets_x, bins=100,density=True)
ax[0].axvspan(offsets_x.min(), offsets_x.max(), color='green', alpha=0.1)
ax[0].set_xlabel('offsets_x')
ax[0].set_ylabel('counts')
ax[0].set_title(f"Mean offset_x is {np.mean(offsets_x):0.3f}, min:{np.min(offsets_x):0.3f}, max:{np.max(offsets_x):0.3f}")

ax[1].hist(offsets_y, bins=100, density=True)
ax[1].axvspan(offsets_y.min(), offsets_y.max(), color='green', alpha=0.1)
ax[1].set_xlabel('offsets_y')
ax[1].set_ylabel('counts')
ax[1].set_title(f"Mean offset_y is {np.mean(offsets_y):0.3f}, min:{np.min(offsets_y):0.3f}, max:{np.max(offsets_y):0.3f}")


fig.suptitle('Offsets predicted by the network')
plt.tight_layout()
plt.show()

In [20]:
filtered_preds = filt_preds_xyz(preds_raw, sigma_x=20, sigma_y=20, sigma_z=40, is_3d=True)

After nms filter.. (135348, 13)


In [16]:
z_values = np.array([x[4] for x in preds_raw])
ph_values = np.array([x[5] for x in preds_raw])

In [17]:
len(z_values), len(ph_values)

(136027, 136027)

In [23]:
plt.figure()
plt.scatter(z_values, ph_values)
plt.show()

In [24]:
plt.figure()
plt.hist(z_values, bins=1000)
plt.show()

In [25]:
len(filtered_preds)

110750

In [26]:
filtered_preds[0]

array([2.0000000e+00, 1.0000000e+00, 1.7790553e+04, 3.2863596e+03,
       3.0838913e+02, 2.2437366e+03, 7.7320665e-01, 1.3856201e+01,
       1.8931519e+01, 3.7946724e+01, 2.9915436e+02, 2.0080650e-01,
       5.9380222e-02], dtype=float32)

In [27]:
z_values_filtered = np.array([x[4] for x in filtered_preds])

In [28]:
z_values_filtered.shape

(110750,)

In [29]:
plt.figure()
plt.hist(z_values_filtered, bins=1000)
plt.show()

In [30]:
np.mean(z_values_filtered)

93.13534

In [31]:
np.std(z_values_filtered)

103.15979

In [33]:
offsets_x = np.array([x[11] for x in filtered_preds])
offsets_y = np.array([x[12] for x in filtered_preds])

In [34]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].hist(offsets_x, bins=100,density=True)
ax[0].axvspan(offsets_x.min(), offsets_x.max(), color='green', alpha=0.1)
ax[0].set_xlabel('offsets_x')
ax[0].set_ylabel('counts')
ax[0].set_title(f"Mean offset_x is {np.mean(offsets_x):0.3f}, min:{np.min(offsets_x):0.3f}, max:{np.max(offsets_x):0.3f}")

ax[1].hist(offsets_y, bins=100, density=True)
ax[1].axvspan(offsets_y.min(), offsets_y.max(), color='green', alpha=0.1)
ax[1].set_xlabel('offsets_y')
ax[1].set_ylabel('counts')
ax[1].set_title(f"Mean offset_y is {np.mean(offsets_y):0.3f}, min:{np.min(offsets_y):0.3f}, max:{np.max(offsets_y):0.3f}")


fig.suptitle('Offsets predicted by the network')
plt.tight_layout()
plt.show()

In [16]:
import csv
csvfile = outputdir+'/'+'cellbgnet_output.csv'
with open(csvfile, 'w') as f:
    table = csv.writer(f)
    header=['molecule','image','x','y','z','photon','prob',\
            'sigma_x','sigma_y','sigma_z','sigma_photon',\
            'x_offset','y_offset','z_offset']
    table.writerow(header)
    for i in range(len(preds_raw)):
        table.writerow(preds_raw[i])
f.close()

### Preds list has the following in order

    1. counter of the molecule 
    2. image number used to index into the number of the image in the prediction arrays
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset
